# Power Series Tools for Differential Equations

In DEs we sometimes find ourselves with an intractable differential equation. In this case we often turn to computer algebra systems to solve. These systems shouldn't be a mystery, but they're often treated as a black box. This notebook will show you the inside of that black box. There's a fair bit of code here, some of it written inefficiently because we're trying to illustrate how a computer algebra system works, not write an excellent one ourselves.  

First, we'll a define a couple utility functions to make our lives easier and our formatting a bit cleaner.

In [254]:
import numpy as np

def format_float(num: float) -> float:
    '''Keeps floats from using scientific notation and removes trailing 0s'''
    return np.format_float_positional(num, trim='-')

def pad_coefficients(coefficients: list, length: int) -> list:
    '''Pads a coefficient list out to a set length by appending zeros.'''
    while len(coefficients) <= length:
        coefficients.append(0)
    return coefficients

Now, we'll need to consider the humble polynomial. A polynomial like $q_1(x) = 2 + 0x + x^2$ will be our starting point. Notice we've written the polynomial backwards from the usual, this is because we'll be moving on to powerseries in a moment and power series are written starting with the constant term. The next big block of code defines the idea of a Polynomial for us in Python. It also tells Python how to add, subtract and multiply polynomials.

In [255]:
class Polynomial():
    def __init__(self, coefficients):
        '''Generates a polynomial from a list of coefficients.'''
        # set degree
        last_nonzero_index = 0
        for i, coef in enumerate(coefficients):
            if coef != 0:
                last_nonzero_index = i
        self.degree = last_nonzero_index

        # set coefficients
        self.coefficients = coefficients
        

    def evalutate_at(self, x: float) -> float:
        '''Evaluates a polynomial at a value.'''
        return sum([coef*x**i for i, coef in enumerate(self.coefficients)])
    
    def __repr__(self):
        '''String representation of a polynomial.'''
        string = ' + '.join([f"{format_float(coef)}x^{i}" for i, coef in enumerate(self.coefficients)])
        return string

    def _repr_latex_(self):
        '''Latex representation of a polynomial. Automatically called by Jupyter.'''
        def format(i, coef):
            '''Format coefficients and indices as monomials'''
            # format_float is to stop scientific notation for aesthetics
            # constant terms shouldn't have xs
            if i == 0:
                return f'{format_float(coef)}'
            # the linear term shouldn't have a power
            elif i == 1:
                return f"{format_float(coef)}x"
            # the rest should be coef*x^i
            else:
                # all the brackets are for latex formatting, the inner pair are for the f string and the doubles come through as a single
                return f"{format_float(coef)}x^{{{i}}}"
        string = ' + '.join([format(i, coef) for i, coef in enumerate(self.coefficients)])
        return f'$${string}$$'
    
    def __add__(self, other):
        '''Add two Polynomials'''
        # find the degree of the result
        result_degree = max(self.degree, other.degree)

        # pad out both lists of coeficients with 0 to reach the degree of the result
        self_coefficients = pad_coefficients(self.coefficients, result_degree)
        other_coefficients = pad_coefficients(other.coefficients, result_degree)

        # combine like terms and return a Polynomial with those coefficients
        added_coefficients = [coef + other_coefficients[i] for i, coef in enumerate(self_coefficients)]
        return Polynomial(added_coefficients)
    
    def __sub__(self, other):
        '''Subtract two Polynomials'''
        # scale the coefficients by -1 and add
        other_coefficients = [-1*coef for coef in other.coefficients]
        return self + Polynomial(other_coefficients)
    
    def __mul__(self, other):
        '''Multiply two Polynomials'''
        # initially start with the zero polynomial
        result = Polynomial([0])

        # For each term in the Polynomial, mulitply it by the second Poly. Add them all and return it.
        for k, self_coef in enumerate(self.coefficients):
            # multiply by x^k (just shift coefficients)
            mult_by_x_k = [0 for _ in range(k)] + other.coefficients

            # multiply the result by the coefficient from the first polynomial
            mult_by_coef = [self_coef*value for value in mult_by_x_k]

            # turn the result into a polynomial and add it to the results so far
            this_row = Polynomial(mult_by_coef)
            result += this_row
        return result
        
        


Here are some polynomials as examples. p1 is $1 + 7x^2 + 9x^3$, which will appear as $1 + 0x + 7x^2 + 9x^3$ and p2 is $2 + x$. We compute the product and sum in the next couple of cells for illustration.

In [256]:
p1 = Polynomial([1, 0, 7, 9])
p2 = Polynomial([2, 1])

test_sum = p1+p2
test_sum

2


3x^0 + 1x^1 + 7x^2 + 9x^3

In [257]:
test_product = p1*p2
test_product

2x^0 + 1x^1 + 14x^2 + 25x^3 + 9x^4

Remember power series from Calc 1? Here we'll be making use of the MacLaurin Series (a Taylor Series centered at 0). Here are a couple formulas you may recognize.

$$e^x = \frac{x^n}{n!}$$  
$$\cos(x) = \frac{(-1)^nx^{2n}}{(2n)!}$$  
$$\sin(x) = \frac{(-1)^nx^{2n+1}}{(2n+1)!}$$

Factorials are really common in power series, so we'll need a helper function so Python can calculate factorials.

In [258]:
def factorial(n):
    if not isinstance(n, int) or n < 0:
        raise TypeError("Factorials are only defined for positive integers")

    if n == 0:
        return 1
    else:
        return n*factorial(n-1)

Now we'll tell Python how a Power Series Approximation works. Basically we'll be treating it as a big polynomial. By default here we'll be limiting ourselves to a degree 10 approximation. Feel free to change that, but be aware that Python can only handle so much without some more serious code work than we've done here. 

In [259]:
class Power_Series_Approximation(Polynomial):
    def __init__(self, formula, degree=10):
        # set the formula for for the power series expansion
        self.formula = formula
        
        # set the degree
        self.degree = degree

        # set coefficients using the formula
        self.coefficients = []
        for i in range(self.degree):
            self.coefficients.append(self.formula(i))
        

    


Here we've converted the Power Series for $e^x$ and $\cos(x)$ into formulas for making Power Series Approximation objects.

In [260]:
def e_series_formula(n):
    return 1/factorial(n)

def cosine_series_formula(n):
    coef = 0
    if n % 2 == 0:
        sign = (-1)**(n-1)/2
        coef = sign/factorial(n)
    return coef

Here we've made a series approximation for $e^x$.

In [265]:

e_series = Power_Series_Approximation(e_series_formula)
e_series
    

1x^0 + 1x^1 + 0.5x^2 + 0.16666666666666666x^3 + 0.041666666666666664x^4 + 0.008333333333333333x^5 + 0.001388888888888889x^6 + 0.0001984126984126984x^7 + 0.0000248015873015873x^8 + 0.0000027557319223985893x^9


This should give you a starting point on a calculator for solving differential equations using power series approximations. A pro tip here, the following cell output isn't easy to copy paste into desmos, but the one after it is.

In [266]:
# not very copy-pastable, but pretty
e_series

1x^0 + 1x^1 + 0.5x^2 + 0.16666666666666666x^3 + 0.041666666666666664x^4 + 0.008333333333333333x^5 + 0.001388888888888889x^6 + 0.0001984126984126984x^7 + 0.0000248015873015873x^8 + 0.0000027557319223985893x^9

In [267]:
# ugly, but copy-pastable
print(e_series)

1x^0 + 1x^1 + 0.5x^2 + 0.16666666666666666x^3 + 0.041666666666666664x^4 + 0.008333333333333333x^5 + 0.001388888888888889x^6 + 0.0001984126984126984x^7 + 0.0000248015873015873x^8 + 0.0000027557319223985893x^9
